## Vit
---

In [2]:
#![feature(portable_simd)]
use std::simd::{u64x64, SimdUint};

In [3]:
#[derive(Debug, Default)]
struct Vit {
    blocks: Vec<u64x64>,
}

In [4]:
/// Vit, a bit vector based on portable simd.
impl Vit {
    fn with_capacity(capacity: &usize) -> Self {
        let num_blocks = (*capacity as f64 / 4096f64).ceil() as usize;
        let blocks = vec![u64x64::splat(0); num_blocks];
        Vit { blocks: blocks }
    }
    fn offsets(offset: &usize) -> (usize, usize, usize) {
        let offset1 = offset / 4096;
        let offset2 = (offset - 4096 * offset1) / 64;
        let offset3 = offset % 64;
        (offset1, offset2, offset3)
    }
    fn set(&mut self, offset: &usize, val: bool) {
        let (offset1, offset2, offset3) = Vit::offsets(offset);
        let bits = self.blocks[offset1][offset2];
        let is_set = (bits & (1 << offset3)) != 0;
        if (is_set & !val) | (!is_set & val) {
            self.blocks[offset1][offset2] = bits ^ (1 << offset3);
        }
    }
    fn check(&self, offset: &usize) -> bool {
        let (offset1, offset2, offset3) = Vit::offsets(offset);
        (self.blocks[offset1][offset2] & (1 << offset3)) != 0
    }
    fn check_all(&self, offsets: &[usize]) -> bool {
        for x in offsets.iter() {
            if !self.check(x) {
                return false;
            }
        }
        true
    }
    fn set_all(&mut self, offsets: &[usize], val: bool) {
        offsets.iter().for_each(|x|self.set(x,val));
    }
    fn size(&self) -> usize {
        self.blocks.len() * 4096
    }
}

In [5]:
let mut v = Vit::with_capacity(&8000);
v

Vit { blocks: [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]] }

In [6]:
v.set_all(&[0, 1, 2, 64, 128, 192, 1000, 6000], true);
v

Vit { blocks: [[7, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1099511627776, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 281474976710656, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]] }

In [7]:
v.check_all(&[0,1,2])

true

In [8]:
v.check_all(&[1,20])

false

In [9]:
v.check_all(&[1000,6000])

true

In [10]:
v.size()

8192

In [11]:
Vit::offsets(&1000)

(0, 15, 40)